In [15]:
# Imports

In [16]:
#import os
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.resources import CDN
from flask import Flask, render_template, request, redirect
import numpy as np 
import pandas as pd
import requests

In [17]:
# Gathering Data 

In [18]:
API_URL = "https://www.alphavantage.co/query" 
symbols =  ['QCOM',"INTC","PDD"]

time_data = {}

for symbol in symbols:
    data = { "function": "TIME_SERIES_MONTHLY", 
    "symbol": symbol,
    "outputsize" : "full",
    "datatype": "json", 
    "apikey": "WNZNE5X63OBPP78N" } 

    response = requests.get(API_URL, data) 
    response_json = response.json()
    #print(response_json)
    data = pd.DataFrame.from_dict(response_json['Monthly Time Series'], orient= 'index').sort_index(axis=1)
    data = data.rename(columns={ '1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
    data = data[[ 'Open', 'High', 'Low', 'Close', 'Volume']]
    data['Date'] = pd.to_datetime(data.index)
    time_data[symbol] = data
    print(data.tail()) # check OK or not # data.tail()

               Open     High      Low    Close     Volume       Date
2020-02-28  85.5500  92.4400  73.3400  78.3000  220392367 2020-02-28
2020-03-31  79.0150  82.6500  58.0000  67.6500  322114348 2020-03-31
2020-04-30  65.0000  80.8700  64.5500  78.6700  215849377 2020-04-30
2020-05-29  76.8500  81.9600  74.0900  80.8800  198186967 2020-05-29
2020-06-26  79.8900  92.1400  79.2400  88.0200  207519690 2020-06-26
               Open     High      Low    Close     Volume       Date
2020-02-28  64.4550  68.0900  53.6000  55.5200  441561383 2020-02-28
2020-03-31  55.7900  59.0200  43.6300  54.1200  961068384 2020-03-31
2020-04-30  52.5000  62.1300  51.2200  59.9800  617700468 2020-04-30
2020-05-29  58.8300  64.1400  56.1000  62.9300  409267014 2020-05-29
2020-06-26  62.4851  65.1100  57.0700  57.5000  516880000 2020-06-26
               Open     High      Low    Close     Volume       Date
2020-02-28  35.0000  37.7200  32.7800  35.7800  156590479 2020-02-28
2020-03-31  37.0000  38.7800  30.2

In [19]:
time_data['INTC'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 2000-07-31 to 2020-06-26
Data columns (total 6 columns):
Open      240 non-null object
High      240 non-null object
Low       240 non-null object
Close     240 non-null object
Volume    240 non-null object
Date      240 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 13.1+ KB


In [20]:
# Function to plot graph

In [21]:
def plot_price(symbol):
    
    #output_file("line.html")

    source = ColumnDataSource(time_data[symbol])
    
    p = figure(x_axis_type = 'datetime') #p = figure(plot_width=400, plot_height=400
    p.line(x = 'Date', y = 'Open', source = source,  legend = symbol + " " + 'Open' , color = 'red' ) 
    p.line(x = 'Date', y = 'High', source = source,  legend = symbol + " " + 'High' , color = 'green')
    p.line(x = 'Date', y = 'Low' , source = source,  legend = symbol + " " + 'Low'  , color = 'yellow') 
    p.line(x = 'Date', y = 'Close',source = source,  legend = symbol + " " + 'Close', color = 'blue') 
    p.yaxis.axis_label = 'Stock Price'
    p.xaxis.axis_label = 'Date'
    
    #html = file_html(p, "line.html")
    #show(p)
    return p

#symbol = 'QCOM' # 'INTC', 'PDD'
#plot_price(symbol)

In [22]:
# Flask app

In [23]:
app = Flask(__name__)
app.vars={}

@app.route('/')
def main():
    return redirect('/home')

@app.route('/home', methods = ['GET','POST'])
def home():
    if request.method=='GET':
        return render_template('basic.html')
    else:
        app.vars['company'] = request.form['options']
        return redirect('/results')

@app.route('/results', methods = ['GET','POST'])
def results():
    if request.method=='GET':
        company = app.vars['company']
        return render_template('results.html', company = company)
    else:
        app.vars['company'] = request.form['options']
        return redirect('/results')

@app.route('/bokehPlot', methods = ['GET'])
def bokehPlot():
    company = app.vars['company']
    script, div = components(plot_price(company))
    return """
    <!doctype html>
    <head>
        {bokeh_css}
    </head>
    <body>
        {div}
        {bokeh_js}
        {script}
    </body>
    """.format(script=script, div=div, bokeh_css=CDN.render_css(), bokeh_js=CDN.render_js())

if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
